In [147]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from soilgasflux_fcs import multiprocess_raw_data, json_reader, fcs
from soilgasflux_fcs import synthetic_postProcessing
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [242]:
# /Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-24
data_folder = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/gals_20250829/sgf_07_pt02/data/2025-08-29')
# data_folder = pathlib.Path(r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/sgf_07_pt02/data/2025-08-23')
a = json_reader.Initializer(folderPath=data_folder)
df = a.prepare_rawdata()

In [243]:
df.dtypes

datetime_utc               object
si_temperature            float64
bmp_temperature           float64
si_humidity               float64
datetime           datetime64[ns]
k30_co2                     int64
bmp_pressure              float64
id                         object
timedelta                   int64
dtype: object

In [244]:
# fig, ax = plt.subplots()
dfs = []
for i in df['id'].unique():
    df_id = df.loc[df['id']==i]
    df_id = df_id.sort_values(by='timedelta')

    df_id.set_index('datetime', inplace=True)
    df_id = df_id.asfreq('S')

    df_id_interpolated = df_id.interpolate(method='linear')
    df_id_interpolated['id'] = df_id_interpolated['id'].fillna(method='ffill')
    df_id_interpolated.reset_index(inplace=True)
    dfs.append(df_id_interpolated)

    # print(df_id_interpolated)
    # break
df_interpolated = pd.concat(dfs)

In [245]:
for i in df_interpolated['id'].unique():
    df_id = df_interpolated.loc[df_interpolated['id']==i]
    

In [246]:
df_interpolated

,datetime,datetime_utc,si_temperature,bmp_temperature,si_humidity,k30_co2,bmp_pressure,id,timedelta
0,2025-08-29 00:13:24,2025-08-29 00:13:24,15.17324,16.49,98.44641,497.0,96215.05,2025-08-29_00-13-24,0.0
1,2025-08-29 00:13:25,2025-08-29 00:13:25,15.19469,16.52,98.47693,496.0,96213.27,2025-08-29_00-13-24,1.0
2,2025-08-29 00:13:26,2025-08-29 00:13:26,15.19469,16.56,98.52271,494.0,96214.42,2025-08-29_00-13-24,2.0
3,2025-08-29 00:13:27,2025-08-29 00:13:27,15.18397,16.59,98.55322,491.0,96214.06,2025-08-29_00-13-24,3.0
4,2025-08-29 00:13:28,2025-08-29 00:13:28,15.20542,16.63,98.58374,489.0,96214.51,2025-08-29_00-13-24,4.0
...,...,...,...,...,...,...,...,...,...
136,2025-08-29 14:19:15,2025-08-29 14:19:15,26.75635,30.39,81.43286,558.0,95824.23,2025-08-29_14-16-59,136.0
137,2025-08-29 14:19:16,2025-08-29 14:19:16,26.78852,30.40,81.43286,559.0,95825.10,2025-08-29_14-16-59,137.0
138,2025-08-29 14:19:17,2025-08-29 14:19:17,26.77779,30.40,81.43286,560.0,95825.89,2025-08-29_14-16-59,138.0
139,2025-08-29 14:19:18,2025-08-29 14:19:18,26.79925,30.41,81.46338,560.0,95827.35,2025-08-29_14-16-59,139.0


In [247]:
df_interpolated.groupby(by='id')['timedelta'].max().min(),df_interpolated.groupby(by='id')['timedelta'].max().max()


(136.0, 141.0)

In [248]:
df_interpolated.loc[df_interpolated['timedelta']>df_interpolated.groupby(by='id')['timedelta'].max().min()] = np.nan
# df_interpolated.loc[df_interpolated['timedelta']>131] = np.nan

In [249]:
df_interpolated.dropna(subset='timedelta', inplace=True)

In [250]:
# df_interpolated.dropna(inplace=True)

In [251]:
df_interpolated['timedelta'].unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136.])

In [252]:
df_interpolated['timedelta'] = df_interpolated['timedelta'].astype(int)

In [253]:
a = multiprocess_raw_data.Multiprocessor()

b = a.run(df=df_interpolated, chamber_id='test01_20250829', output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results')
# b = a.run(df=df_interpolated, chamber_id='test01_20250823', output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/low-cost_sensor/gals_20250829/results')

Multiprocessing started
CPU core count: 14


Date: 2025-08-29
Processing ID: 2025-08-29_05-02-55
Processing ID: 2025-08-29_00-13-24
Processing ID: 2025-08-29_04-00-58
Processing ID: 2025-08-29_02-38-05
Processing ID: 2025-08-29_00-54-48
Processing ID: 2025-08-29_02-48-33
Processing ID: 2025-08-29_02-07-07
Processing ID: 2025-08-29_01-25-40
Processing ID: 2025-08-29_04-31-57
Processing ID: 2025-08-29_00-23-53
Processing ID: 2025-08-29_05-13-24
Processing ID: 2025-08-29_01-36-09
Processing ID: 2025-08-29_03-19-31
Processing ID: 2025-08-29_03-50-29
Results: dict_keys(['2025-08-29_00-54-48'])
Processing ID: 2025-08-29_00-59-59
Results: dict_keys(['2025-08-29_00-59-59'])
Processing ID: 2025-08-29_05-44-22
Error in fitting the target function:
The model function generated NaN values and the fit aborted! Please check your model function and/or set boundaries on parameters where applicable. In cases like this, using "nan_policy='omit'" will probably not work.
ERROR HM ####
'NoneType' object is not subscriptable
Results: dict_keys(['2025-

In [254]:
chamber_id='test8kMC_20250829'


In [255]:
# chamber_id='test8kMC_20250823'
b = a.run_MC(df=df_interpolated, 
             chamber_id=chamber_id, 
             output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results')

Multiprocessing started
CPU core count: 14
Date: 2025-08-29
Processing ID: 2025-08-29_03-19-31
Processing ID: 2025-08-29_01-25-40
Processing ID: 2025-08-29_00-54-48
Processing ID: 2025-08-29_02-07-07
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 60
Processing ID: 2025-08-29_03-50-29
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 70
Processing ID: 2025-08-29_02-38-05
Processing ID: 2025-08-29_00-13-24
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 80
Processing ID: 2025-08-29_02-48-33
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 90
Processing ID: 2025-08-29_05-02-55
Processing ID: 2025-08-29_04-31-57
Processing ID: 2025-08-29_00-23-53
Processing ID: 2025-08-29_05-13-24
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 100
Processing ID: 2025-08-29_04-00-58
Processing ID: 2025-08-29_01-36-09
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 110
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cuto

Traceback (most recent call last):
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/fcs.py", line 141, in run_MC
    hm_results = hm.calculate_MC(deadband=deadband, cutoff=cutoff, n=n_MC)
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/hm_model.py", line 247, in calculate_MC
    sampler, flat_samples, logprob_samples = mcmc.run_mcmc(t=self.timestamp.values[deadband:cutoff],
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/mcmc.py", line 65, in run_mcmc
    pos_cx = np.exp(np.random.uniform(low=np.log(cx_bf*1e-2),
  File "numpy/random/mtrand.pyx", line 1156, in numpy.random.mtrand.RandomState.uniform
OverflowError: Range exceeds valid bounds
Traceback (most recent call last):
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/fcs.py", line 141, in run_MC
    hm_results = hm.calculate_MC(deadband=deadband, cutoff=cutoff, n=n_MC)
  File

ERROR HM ####
Range exceeds valid bounds
Deadband: 10, Cutoff: 110
ERROR HM ####
Range exceeds valid bounds
Deadband: 10, Cutoff: 120
ERROR HM ####
Range exceeds valid bounds
Deadband: 10, Cutoff: 130
ERROR HM ####
Range exceeds valid bounds
Deadband: 20, Cutoff: 80
ERROR HM ####
Range exceeds valid bounds
Deadband: 20, Cutoff: 90
ERROR HM ####
Range exceeds valid bounds
Deadband: 20, Cutoff: 100
ERROR HM ####
Range exceeds valid bounds
Deadband: 20, Cutoff: 110
ERROR HM ####
Range exceeds valid bounds
Deadband: 20, Cutoff: 120
ERROR HM ####
Range exceeds valid bounds
Deadband: 20, Cutoff: 130
ERROR HM ####
Range exceeds valid bounds
Deadband: 30, Cutoff: 90
ERROR HM ####
Range exceeds valid bounds
Deadband: 30, Cutoff: 100
ERROR HM ####
Range exceeds valid bounds
Deadband: 30, Cutoff: 110
ERROR HM ####
Range exceeds valid bounds
Deadband: 30, Cutoff: 120
ERROR HM ####
Range exceeds valid bounds
Deadband: 30, Cutoff: 130


Traceback (most recent call last):
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/fcs.py", line 141, in run_MC
    hm_results = hm.calculate_MC(deadband=deadband, cutoff=cutoff, n=n_MC)
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/hm_model.py", line 247, in calculate_MC
    sampler, flat_samples, logprob_samples = mcmc.run_mcmc(t=self.timestamp.values[deadband:cutoff],
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/mcmc.py", line 65, in run_mcmc
    pos_cx = np.exp(np.random.uniform(low=np.log(cx_bf*1e-2),
  File "numpy/random/mtrand.pyx", line 1156, in numpy.random.mtrand.RandomState.uniform
OverflowError: Range exceeds valid bounds
Traceback (most recent call last):
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/fcs.py", line 141, in run_MC
    hm_results = hm.calculate_MC(deadband=deadband, cutoff=cutoff, n=n_MC)
  File

ERROR HM ####
Range exceeds valid bounds
Deadband: 40, Cutoff: 100
ERROR HM ####
Range exceeds valid bounds
Deadband: 40, Cutoff: 110
ERROR HM ####
Range exceeds valid bounds
Deadband: 40, Cutoff: 120
ERROR HM ####
Range exceeds valid bounds
Deadband: 40, Cutoff: 130
ERROR HM ####
Range exceeds valid bounds
Deadband: 50, Cutoff: 110
ERROR HM ####
Range exceeds valid bounds
Deadband: 50, Cutoff: 120
ERROR HM ####
Range exceeds valid bounds
Deadband: 50, Cutoff: 130
Results: dict_keys(['2025-08-29_00-54-48'])
Processing ID: 2025-08-29_00-59-59
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 60
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 70
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 80
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 90
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 100
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff: 110
ERROR HM ####
Range exceeds valid bounds
Deadband: 0, Cutoff

...

Results: dict_keys(['2025-08-29_01-36-09'])
Processing ID: 2025-08-29_02-01-52
Results: dict_keys(['2025-08-29_02-07-07'])
Processing ID: 2025-08-29_02-12-21
Processing ID: 2025-08-29_06-15-20
Results: dict_keys(['2025-08-29_02-38-05'])
Processing ID: 2025-08-29_02-43-19
Results: dict_keys(['2025-08-29_03-19-31'])
Processing ID: 2025-08-29_03-24-46
Results: dict_keys(['2025-08-29_05-02-55'])
Processing ID: 2025-08-29_05-08-10
Results: dict_keys(['2025-08-29_03-50-29'])
Processing ID: 2025-08-29_03-55-44
Results: dict_keys(['2025-08-29_02-48-33'])
Processing ID: 2025-08-29_03-14-17
Error in fitting the target function:
The model function generated NaN values and the fit aborted! Please check your model function and/or set boundaries on parameters where applicable. In cases like this, using "nan_policy='omit'" will probably not work.
ERROR HM ####
'NoneType' object is not subscriptable
Deadband: 50, Cutoff: 130
Results: dict_keys(['2025-08-29_01-25-40'])
Processing ID: 2025-08-29_01-30-5

Traceback (most recent call last):
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/fcs.py", line 141, in run_MC
    hm_results = hm.calculate_MC(deadband=deadband, cutoff=cutoff, n=n_MC)
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/hm_model.py", line 221, in calculate_MC
    cx = result_fit['parameters_best_fit']['cx']
TypeError: 'NoneType' object is not subscriptable


Results: dict_keys(['2025-08-29_05-13-24'])
Processing ID: 2025-08-29_05-39-08
Results: dict_keys(['2025-08-29_00-13-24'])
Processing ID: 2025-08-29_00-18-39
Error in fitting the target function:
The model function generated NaN values and the fit aborted! Please check your model function and/or set boundaries on parameters where applicable. In cases like this, using "nan_policy='omit'" will probably not work.
ERROR HM ####
'NoneType' object is not subscriptable
Deadband: 50, Cutoff: 120
Error in fitting the target function:
The model function generated NaN values and the fit aborted! Please check your model function and/or set boundaries on parameters where applicable. In cases like this, using "nan_policy='omit'" will probably not work.
ERROR HM ####
'NoneType' object is not subscriptable
Deadband: 50, Cutoff: 130
Results: dict_keys(['2025-08-29_02-01-52'])


Traceback (most recent call last):
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/fcs.py", line 141, in run_MC
    hm_results = hm.calculate_MC(deadband=deadband, cutoff=cutoff, n=n_MC)
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/hm_model.py", line 221, in calculate_MC
    cx = result_fit['parameters_best_fit']['cx']
TypeError: 'NoneType' object is not subscriptable
Traceback (most recent call last):
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/fcs.py", line 141, in run_MC
    hm_results = hm.calculate_MC(deadband=deadband, cutoff=cutoff, n=n_MC)
  File "/Users/alexnaokiasatokobayashi/git/soilgasflux_fcs/notebooks/../soilgasflux_fcs/hm_model.py", line 221, in calculate_MC
    cx = result_fit['parameters_best_fit']['cx']
TypeError: 'NoneType' object is not subscriptable


Processing ID: 2025-08-29_06-25-49
Results: dict_keys(['2025-08-29_06-15-20'])
Processing ID: 2025-08-29_06-20-35
Results: dict_keys(['2025-08-29_05-08-10'])
Results: dict_keys(['2025-08-29_04-26-42'])
Processing ID: 2025-08-29_06-56-48
Processing ID: 2025-08-29_07-27-46
Results: dict_keys(['2025-08-29_02-12-21'])
Processing ID: 2025-08-29_07-38-15
Results: dict_keys(['2025-08-29_03-24-46'])
Results: dict_keys(['2025-08-29_01-30-55'])
Processing ID: 2025-08-29_08-09-14
Processing ID: 2025-08-29_08-40-12
Results: dict_keys(['2025-08-29_02-43-19'])
Results: dict_keys(['2025-08-29_03-14-17'])
Processing ID: 2025-08-29_08-50-41
Results: dict_keys(['2025-08-29_05-49-36'])
Processing ID: 2025-08-29_09-21-40
Results: dict_keys(['2025-08-29_03-55-44'])
Processing ID: 2025-08-29_09-52-39
Results: dict_keys(['2025-08-29_04-37-11'])
Processing ID: 2025-08-29_10-03-09
Results: dict_keys(['2025-08-29_00-18-39'])
Processing ID: 2025-08-29_10-34-09
Processing ID: 2025-08-29_11-05-09
Results: dict_key

In [256]:
ds_test = xr.open_dataset(r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results/test8kMC_20250829_2025-08-29.nc')
test = a.select_bestPareto(ds=ds_test, 
                           chamber_id=chamber_id, 
                           output_folder=r'/Users/alexnaokiasatokobayashi/Documents/data/Gals/results')

Selecting best pareto front from dataset
Pareto front not found for time: 2025-08-29T00:49:36.000000000
Pareto front not found for time: 2025-08-29T00:54:48.000000000
Pareto front not found for time: 2025-08-29T00:59:59.000000000
Best dcdt shape: (69, 8000)
Best pareto dataset saved to: /Users/alexnaokiasatokobayashi/Documents/data/Gals/results
